In [13]:
import numpy as np
import pandas as pd
import matplotlib as plt
import sklearn
import json
import ast

In [15]:
df = pd.read_csv("../data/play_by_play_fbs_regular_all.csv")

In [16]:
np.random.seed(12)

In [17]:
sample = df.sample(100_000).copy()

In [18]:
sample.head()

,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,game_id,...,defense_timeouts,yard_line,yards_to_goal,down,distance,scoring,yards_gained,play_type,play_text,ppa
973573,273070030190,Oregon State,Pac-10,USC,Pac-10,USC,Oregon State,3,24,273070030,...,NaN,86,86,3,9,False,18,Rush,Sean Canfield rush for 18 yards to the OrgSt 4...,1.728350
2053346,332572483166,Tennessee,SEC,Oregon,Pac-12,Oregon,Tennessee,7,45,332572483,...,3.0,43,43,4,9,False,0,Punt,"Michael Palardy punt for 32 yards, downed at t...",NaN
533251,272440356197,Missouri,Big 12,Illinois,Big Ten,Illinois,Missouri,40,34,272440356,...,NaN,73,73,3,8,False,0,Sack,Chase Daniel sacked by Justin Sanders at the M...,-0.224210
991523,282570328044,Utah,Mountain West,Utah State,Western Athletic,Utah State,Utah,7,7,282570328,...,NaN,28,28,1,10,False,0,Sack,Brian Johnson sacked for a loss of 11 yards to...,-0.512082
318622,272930356080,Illinois,Big Ten,Michigan,Big Ten,Illinois,Michigan,14,10,272930356,...,NaN,43,57,3,2,False,0,Rush,Juice Williams rush for no gain to the Illin 43.,-0.912740


In [19]:
def time_remaining(row):
    period = int(row.period)
    time = ast.literal_eval(row.clock)
    remaining = ((4 - period) * 15 * 60) + (time['minutes'] * 60) + time['seconds']

sample['time_remaining'] = sample[['period', 'clock']].apply(time_remaining, axis=1)


In [23]:
sample.head()
sample.columns

Index(['id', 'offense', 'offense_conference', 'defense', 'defense_conference',
       'home', 'away', 'offense_score', 'defense_score', 'game_id', 'drive_id',
       'drive_number', 'play_number', 'period', 'clock', 'offense_timeouts',
       'defense_timeouts', 'yard_line', 'yards_to_goal', 'down', 'distance',
       'scoring', 'yards_gained', 'play_type', 'play_text', 'ppa',
       'time_remaining'],
      dtype='object')

In [25]:
def yards_from_own_goal_line(row):
    yards = int(row.yards_to_goal)
    yards_from_own_goal = 100-yards
    return yards_from_own_goal
sample["yards_from_own_goal_line"] = 100-sample["yards_to_goal"]

In [26]:
def yards_to_go_for_first_down(row):
    return row.distance
sample["yards_to_go_for_first_down"] = sample["distance"]

In [27]:
sample.head()

,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,game_id,...,down,distance,scoring,yards_gained,play_type,play_text,ppa,time_remaining,yards_from_own_goal_line,yards_to_go_for_first_down
973573,273070030190,Oregon State,Pac-10,USC,Pac-10,USC,Oregon State,3,24,273070030,...,3,9,False,18,Rush,Sean Canfield rush for 18 yards to the OrgSt 4...,1.728350,None,14,9
2053346,332572483166,Tennessee,SEC,Oregon,Pac-12,Oregon,Tennessee,7,45,332572483,...,4,9,False,0,Punt,"Michael Palardy punt for 32 yards, downed at t...",NaN,None,57,9
533251,272440356197,Missouri,Big 12,Illinois,Big Ten,Illinois,Missouri,40,34,272440356,...,3,8,False,0,Sack,Chase Daniel sacked by Justin Sanders at the M...,-0.224210,None,27,8
991523,282570328044,Utah,Mountain West,Utah State,Western Athletic,Utah State,Utah,7,7,282570328,...,1,10,False,0,Sack,Brian Johnson sacked for a loss of 11 yards to...,-0.512082,None,72,10
318622,272930356080,Illinois,Big Ten,Michigan,Big Ten,Illinois,Michigan,14,10,272930356,...,3,2,False,0,Rush,Juice Williams rush for no gain to the Illin 43.,-0.912740,None,43,2
